In [1]:
# conda install -c cyclus java-jdk
'''
pip install pip-tools
pip-compile --generate-hashes
pip-sync
'''
import os
import glob
import shutil
import zipfile
import pandas as pd
from pathlib import Path

from lib.prepro import Preprocessing, SparkDataFrame
import dask
from dask.delayed import delayed
import dask.bag as db

from dask.distributed import Client, LocalCluster

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import StringType, LongType

from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

import findspark
findspark.init()

In [2]:
# PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.원천데이터'
PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터/*'
PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/*'
PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터/*'

prepro = Preprocessing()
sparkdf = SparkDataFrame()

cctv_data = prepro.get_all_file_paths(PATH_CCTV_DATA)
sand_data = prepro.get_all_file_paths(PATH_SAND_DATA)
sand_label_data = prepro.get_all_file_paths(PATH_SAND_LABEL)

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = sparkdf.get_spark_dataframe(sand_label_data)

In [5]:
df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+-------+-----------+---------------------+---------+
|full_path                                                                                                                    |file_id|file_name  |folder_name          |file_size|
+-----------------------------------------------------------------------------------------------------------------------------+-------+-----------+---------------------+---------+
|G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\TL_2.모래위험광물분류\001013.json|001013 |001013.json|TL_2.모래위험광물분류|132984   |
|G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\TL_2.모래위험광물분류\001002.json|001002 |001002.json|TL_2.모래위험광물분류|226203   |
|G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\TL_2.모래위험광물분류\001030.json|001030 |001030.json|TL_2.모래위험광물분류|107594   |
|G:/industry_data/264.건설 모래 품질 관리데이터/01-

In [13]:
os.path.abspath('./')

'f:\\industry\\git_prepro\\pro_prepro'

In [8]:
df.columns

['full_path', 'file_id', 'file_name', 'folder_name', 'file_size']

In [11]:
@udf(StringType())
def extract_file_id(paths):
    '''file_id 추출'''
    file_name = os.path.basename(paths)
    file_id = os.path.splitext(file_name)[0]
    return file_id

In [14]:
from tqdm import tqdm

In [ ]:
dfs = []
for json_data in json_meta_data:
    # json_data를 Spark DataFrame으로 변환
    df = spark.read.json(spark.sparkContext.parallelize([json_data]))
    dfs.append(df)

# 모든 DataFrame을 하나로 합칩니다
# monotonically_increasing_id 함수를 사용하여 각 행에 고유 ID를 할당합니다
# ignore_index=True와 같은 효과를 내기 위해 monotonically_increasing_id 함수 사용
combined_df = None
for df in dfs:
    df = df.withColumn("index", monotonically_increasing_id())
    if combined_df is None:
        combined_df = df
    else:
        combined_df = combined_df.union(df)

# index 열을 제거하여 최종 데이터프레임을 만듭니다
combined_df = combined_df.drop("index")

In [18]:
sand_label_data[3]

'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\\TL_2.모래위험광물분류\\001003.json'

In [30]:
dfs = [spark.read.json(file) for file in sand_label_data[:2]]

In [42]:
sc = spark.sparkContext

In [47]:
sand_label_data[0]

'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\\TL_2.모래위험광물분류\\001013.json'

In [44]:
sp_json = spark.read.json(sand_label_data[0])
sp_json.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



In [49]:
import json
with open(sand_label_data[0], 'r', encoding='utf-8') as f:
    data = json.load(f)
    data_df = pd.json_normalize(data)

In [50]:
data

{'id': '001013.png',
 'image': {'name': '001013.png', 'size': {'width': 2592, 'height': 1944}},
 'dataset': {'type': 1,
  'number': 34,
  'year': 2021,
  'address1': '38°00\'21.465", 127°03\'41.402"',
  'address2': '경기도 연천군 군남면 남계리',
  'sort': '하천',
  'camera': 'SZ61TR',
  'lighting': '20,000룩스'},
 'shapes': [{'name': 'POLYGON',
   'type': '석영',
   'minDotCount': 0,
   'maxDotCount': 100,
   'vertices': [{'x': 355.0401138716355, 'y': 1047.2702635211215},
    {'x': 339.4327076749845, 'y': 1052.236256401874},
    {'x': 329.50072191347925, 'y': 1056.4928217282336},
    {'x': 318.8593085975808, 'y': 1065.7153799353455},
    {'x': 313.8933157168282, 'y': 1075.6473656968508},
    {'x': 308.9273228360756, 'y': 1088.4170616759288},
    {'x': 306.08961261850266, 'y': 1103.3150403181867},
    {'x': 303.961329955323, 'y': 1119.631874069231},
    {'x': 303.25190240092974, 'y': 1133.1109976027024},
    {'x': 303.961329955323, 'y': 1148.0089762449602},
    {'x': 303.961329955323, 'y': 1160.069244669

In [51]:
data_df

,id,shapes,image.name,image.size.width,image.size.height,dataset.type,dataset.number,dataset.year,dataset.address1,dataset.address2,dataset.sort,dataset.camera,dataset.lighting
0,001013.png,"[{'name': 'POLYGON', 'type': '석영', 'minDotCoun...",001013.png,2592,1944,1,34,2021,"38°00'21.465"", 127°03'41.402""",경기도 연천군 군남면 남계리,하천,SZ61TR,"20,000룩스"


In [28]:
from pyspark.sql.functions import monotonically_increasing_id

In [31]:
combined_df = None
for df in dfs:
    df = df.withColumn("index", monotonically_increasing_id())
    if combined_df is None:
        combined_df = df
    else:
        combined_df = combined_df.union(df)

In [4]:
json_data = prepro.normalize_json(sand_label_data)

TypeError: Preprocessing.normalize_json() missing 1 required positional argument: 'json_files'

In [ ]:
json_data

In [ ]:
json_data.show()

In [ ]:
df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+-------+-----------+---------------------+---------+
|full_path                                                                                                                    |file_id|file_name  |folder_name          |file_size|
+-----------------------------------------------------------------------------------------------------------------------------+-------+-----------+---------------------+---------+
|G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\TL_2.모래위험광물분류\001013.json|001013 |001013.json|TL_2.모래위험광물분류|132984   |
|G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\TL_2.모래위험광물분류\001002.json|001002 |001002.json|TL_2.모래위험광물분류|226203   |
|G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터\TL_2.모래위험광물분류\001030.json|001030 |001030.json|TL_2.모래위험광물분류|107594   |
|G:/industry_data/264.건설 모래 품질 관리데이터/01-